In [ ]:
!pip install sec-edgar-downloader

In [ ]:
from sec_edgar_downloader import Downloader
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dl = Downloader("jpark", "jparkcareers@gmail.com", "/content/drive/MyDrive/sec_filings")

In [ ]:
ticker = "MSFT"
start_year = 1995
end_year = 2023

In [ ]:
for year in range(start_year, end_year + 1):
    dl.get("10-K", ticker, after=f"{year}-01-01", before=f"{year}-12-31")

In [ ]:
downloaded_files = os.listdir("/content/drive/MyDrive/sec_filings")
print("Downloaded files:")
for file in downloaded_files:
    print(file)

Downloaded files:
sec-edgar-filings
appl 2006 - 2023.json
sec & sentiment.ipynb


In [ ]:
!pip install beautifulsoup4

In [ ]:
import os
from bs4 import BeautifulSoup
import re
!pip install pandas
# import pandas
import pandas as pd

In [ ]:
base_dir = f"/content/drive/MyDrive/sec_filings/sec-edgar-filings/{ticker}/10-K"

In [ ]:
# rename folders to their respective years
for folder in os.listdir(base_dir):
    if folder.startswith("0"):
        accession_number = folder.split("-")
        if len(accession_number) > 1:
            year = accession_number[1][:2]
            if year.isdigit():
                year = "20" + year if int(year) <= 23 else "19" + year
                new_folder_name = os.path.join(base_dir, year)
                old_folder_path = os.path.join(base_dir, folder)
                os.rename(old_folder_path, new_folder_name)


In [ ]:
def extract_items(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        raw_10k = content
        soup = BeautifulSoup(content, 'lxml')

        # Regex to find <DOCUMENT> tags
        doc_start_pattern = re.compile(r'<DOCUMENT>')
        doc_end_pattern = re.compile(r'</DOCUMENT>')
        # Regex to find <TYPE> tag preceding any characters, terminating at new line
        type_pattern = re.compile(r'<TYPE>[^\n]+')

        # Create 3 lists with the span indices for each regex
        doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
        doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
        doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

        document = {}

        # Create a loop to go through each section type and save only the 10-K section in the dictionary
        for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
            if doc_type == '10-K':
                document[doc_type] = raw_10k[doc_start:doc_end]

        # Write the regex
        regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|1B|7A|7|8))')

        # Use finditer to match the regex
        matches = regex.finditer(document['10-K'])

        # Create the dataframe
        test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

        test_df.columns = ['item', 'start', 'end']
        test_df['item'] = test_df.item.str.lower()

        # Get rid of unnecessary characters from the dataframe
        test_df.replace('&#160;', ' ', regex=True, inplace=True)
        test_df.replace('&nbsp;', ' ', regex=True, inplace=True)
        test_df.replace(' ', '', regex=True, inplace=True)
        test_df.replace('\.', '', regex=True, inplace=True)
        test_df.replace('>', '', regex=True, inplace=True)

        # Drop duplicates
        pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

        # Set item as the dataframe index
        pos_dat.set_index('item', inplace=True)

        # Get Item 1a
        item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item1b']]
        item_1a_content = BeautifulSoup(item_1a_raw, 'lxml')
        item_1a_text = item_1a_content.get_text("\n\n")

        # Get Item 7
        item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]
        item_7_content = BeautifulSoup(item_7_raw, 'lxml')
        item_7_text = item_7_content.get_text("\n\n")

        # Get Item 7a
        item_7a_raw = document['10-K'][pos_dat['start'].loc['item7a']:pos_dat['start'].loc['item8']]
        item_7a_content = BeautifulSoup(item_7a_raw, 'lxml')
        item_7a_text = item_7a_content.get_text("\n\n")

        # Create a dictionary to store the extracted items
        extracted_items = {
            'item1a': item_1a_text,
            'item7': item_7_text,
            'item7a': item_7a_text
        }

        return extracted_items

In [ ]:

extracted_items = {}
year = 1995
folder = str(year)
file_path = f"{base_dir}/{folder}/full-submission.txt"
print("File path:", file_path)

if os.path.exists(file_path):
  extracted_items[year] = extract_items(file_path)
else:
  print(f"File not found for year {year}")

print(extracted_items)

File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/1995/full-submission.txt


KeyError: 'item1a'

In [ ]:
# base_dir = f"/content/drive/MyDrive/sec_filings/sec-edgar-filings/{ticker}/10-K"
extracted_items = {}

# specify the directory path
directory = f"/content/drive/MyDrive/sec_filings/sec-edgar-filings/{ticker}/10-K/1995"


# fet the list of files and directories in the specified directory
files = os.listdir(directory)

# print the list of files and directories
print("Files and directories in the directory:")
for file in files:
    print(file)

Files and directories in the directory:
full-submission.txt


In [ ]:
import logging

# configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# base_dir = "/content/drive/MyDrive/sec_filings/sec-edgar-filings/AAPL/10-K"
print("Base directory:", base_dir)

years = [1995, 1996, 1997, 1999, 2000, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
         2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

extracted_items = {}

for year in years:
    folder = str(year)
    file_path = f"{base_dir}/{folder}/full-submission.txt"
    print("File path:", file_path)

    if os.path.exists(file_path):
        try:
            extracted_items[year] = extract_items(file_path)
            logging.info(f"Successfully extracted items for year {year}")
        except Exception as e:
            logging.error(f"Error occurred while processing file for year {year}: {str(e)}")
    else:
        print(f"File not found for year {year}")

print(extracted_items)

ERROR:root:Error occurred while processing file for year 1995: 'item1a'
ERROR:root:Error occurred while processing file for year 1996: Length mismatch: Expected axis has 0 elements, new values have 3 elements
ERROR:root:Error occurred while processing file for year 1997: Length mismatch: Expected axis has 0 elements, new values have 3 elements
ERROR:root:Error occurred while processing file for year 1999: Length mismatch: Expected axis has 0 elements, new values have 3 elements
ERROR:root:Error occurred while processing file for year 2000: Length mismatch: Expected axis has 0 elements, new values have 3 elements


Base directory: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/1995/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/1996/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/1997/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/1999/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2000/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2002/full-submission.txt


ERROR:root:Error occurred while processing file for year 2002: 'item1a'


File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2003/full-submission.txt


ERROR:root:Error occurred while processing file for year 2003: 'item1a'


...

ERROR:root:Error occurred while processing file for year 2004: 'item1a'


File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2005/full-submission.txt


ERROR:root:Error occurred while processing file for year 2005: 'item1a'


File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2006/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2007/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2008/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2009/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2010/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2011/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2012/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2013/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2014/full-submission.txt
File path: /content/drive/MyDrive/sec_filings/sec-edgar-filings/MSFT/10-K/2015/full-submission.txt
File path:

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
for year, items in extracted_items.items():
    print(f"Year: {year}")
    if items:
        for item_name, item_text in items.items():
            print(f"  {item_name.upper()}:")
            if item_text:
                print(f"    {item_text.strip()[:100]}...")  # Print the first 100 characters of the item text
            else:
                print("    No item text found.")
            print()
    else:
        print("  No items found.")
    print("=" * 50)

In [ ]:
import json


# print the summary of extracted items
print("\nSummary of Extracted Items:")
for year, items in extracted_items.items():
    if items:
        print(f"Year: {year}")
        print("  Extracted Items:")
        for item_name in items.keys():
            print(f"    - {item_name}")
    else:
        print(f"Year: {year} - No items found.")

# save the extracted items to a JSON file
output_file = "/content/drive/MyDrive/sec_filings/extracted_items.json"
with open(output_file, "w") as json_file:
    json.dump(extracted_items, json_file, indent=4)

print(f"\nExtracted items saved to: {output_file}")


Summary of Extracted Items:
Year: 2006
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2007
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2008
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2009
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2010
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2011
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2012
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2013
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2014
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2015
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2016
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2017
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2018
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2019
  Extracted Items:
    - item1a
    - item7
    - item7a
Year: 2020
  Extrac

In [ ]:
def print_dict_structure(d, indent=0):
    for key, value in d.items():
        print('  ' * indent + str(key))
        if isinstance(value, dict):
            print_dict_structure(value, indent+1)

print_dict_structure(extracted_items)


2006
  item1a
  item7
  item7a
2007
  item1a
  item7
  item7a
2008
  item1a
  item7
  item7a
2009
  item1a
  item7
  item7a
2010
  item1a
  item7
  item7a
2011
  item1a
  item7
  item7a
2012
  item1a
  item7
  item7a
2013
  item1a
  item7
  item7a
2014
  item1a
  item7
  item7a
2015
  item1a
  item7
  item7a
2016
  item1a
  item7
  item7a
2017
  item1a
  item7
  item7a
2018
  item1a
  item7
  item7a
2019
  item1a
  item7
  item7a
2020
  item1a
  item7
  item7a
2021
  item1a
  item7
  item7a
2022
  item1a
  item7
  item7a
2023
  item1a
  item7
  item7a


In [ ]:
import requests
import json

# function to extract an item from a filing using the SEC API
def extract_item_from_filing(url, item, api_key):
    api_url = "https://api.sec-api.io/extractor"
    params = {
        'url': url,
        'item': item,
        'type': 'text',
        'token': api_key
    }
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        return "Section not found"

# main function to process filings for a company
def process_filings(filing_urls, items, api_key):
    filings_dict = {}
    for year, url in filing_urls.items():
        filings_dict[year] = {}
        for item in items:
            extracted_text = extract_item_from_filing(url, item, api_key)
            filings_dict[year][f"item{item}"] = extracted_text
    return filings_dict

# API key and filing information
api_key = 'YOUR_API_KEY_HERE'
aapl_filing_urls = {
    '1995': 'URL_FOR_1995',
    # add other years and URLs
}
msft_filing_urls = {
    '1995': 'URL_FOR_1995',
    # add other years and URLs
}
items = ['1', '1A', '7']  # items to extract

# fetch and organize data
aapl_filings = process_filings(aapl_filing_urls, items, api_key)
msft_filings = process_filings(msft_filing_urls, items, api_key)

# dump data to JSON files
with open('aapl_filings.json', 'w') as f:
    json.dump(aapl_filings, f)
with open('msft_filings.json', 'w') as f:
    json.dump(msft_filings, f)
